In [3]:
# Check if GPU device is available
import tensorflow as tf

C:\Users\com\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\framework\dtypes.py:511: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

tensorflow 2.5.0으로 낮춰 잡음  
-> numpy만 upgrade

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# import tensorflow libraries
import tensorflow as tf
# load the easy execution library
# eager execution 라이브러리 로드 
import time

# enable easger execution - this should be done at start of program
# tf.enable_eager_execution()
tf.executing_eagerly()
print("Eager execution: {}".format(tf.executing_eagerly()))

# load the model from keras with imagenet weights
vgg19 = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
vgg19.trainable = False
vgg19.summary()

Eager execution: True
80142336/80134624 [==============================] - 3s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
________________________________________

tf 2.0부터는 eager execution이 기본 모드이다.  
따라서 eager execution mode가 기본이라 이런 절차 및 tensorflow.contrib.eager as tfe가 필요 없다.  

다음은 스타일 및 콘텐츠 등으로 사용할 특정 특성 층들을 선택한다. 

이 층들은 VGG19 모델이 이미지에서 학습한 특성들을 추출하는 데 사용할 것  
이 특성들은 각 이미지의 콘텐츠 및 스타일에 대한 아이디어를 줄 것 

최적화를 수행할 것이므로  
우리의 목적 : 스타일 및 콘텐츠 거리(cost)를 최소화시키는 것 

이 층들을 위의 요약 정보에 나타난 층 이름을 보면서 선택해 보자.  
다른 층들로 선택할 수도 있음   
  
- 콘텐츠 비교 : block_5 conv 층을 사용 
- 스타일 비교 : 여러 개의 conv 층 사용 

이제는 더 이상 모델에 의한 예측에 관심 두지 않는다. 

In [ ]:
# 특성 맵들을 끌어낼 콘텐츠 층 
content_layers = ['block5_conv2']

# 관심 대상인 스타일 층 
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1',
]

# 스타일 및 콘텐츠 층의 수 계산 
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

# 스타일 및 콘텐츠 층에 해당하는 출력층 가져오기 
style_outputs = [vgg19.get_layer(name).output for name in style_layers]
content_outputs = [vgg19.get_layer(name).output for name in content_layers]
model_outputs = style_outputs + content_outputs

# 모델 구축 
style_model = tf.keras.models.Model(vgg19.input, model_outputs)

콘텐츠에 필요한 파일 : 배경 없는 인물 사진, 예제의 사진이 없어서 정국으로 대체,,,  
스타일에 필요한 파일 : 반 고흐의 별이 빛나는 밤에 사진,,, 예제 사진 없어서 찾아서 대체 

In [ ]:
# 그림 그리는 라이브러리 로드  
import matplotlib.pyplot as plt 
%matplotlib inline 

# numpy import 
import numpy as np 
# 이미지를 준비하기 위한 선처리 함수 로드 
from keras.preprocessing import image 
from keras.applications.vgg19 import preprocess_input

ImportError: Matplotlib requires numpy>=1.21; you have 1.19.5